# Testing the API

Satellite data starts being recored on {'date': '2015-06-13'}

In [1]:
import requests
import os
from IPython.display import Image

In [ ]:
# All Data
url = 'https://epic.gsfc.nasa.gov/api/natural/all'

# Most Recent
url = 'https://epic.gsfc.nasa.gov/api/natural/'

# URL by date
date = '20180428'
url = f'https://epic.gsfc.nasa.gov/api/enhanced/date/{date}'
res = requests.get(url)

In [ ]:
# Example data retrieved
print(f'There are {len(res.json())} images available')
res.json()[0]

In [ ]:
# Available data time and coords
fetch = 3
points = []
for e in res.json()[:fetch]:
    points.append((e['image'], e['centroid_coordinates'], e['date']))
points

In [ ]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/{size['med']}/{res.json()[0]['image']}.{size['med']}"
print(img)
#display(Image(url=img))

In [ ]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
for e in res.json():
    img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/png/{e['image']}.{size['big']}"
    #display(Image(url=img))

In [ ]:
# Example data retrieved
print(res.json()[0]['identifier'])
YYYYMMDD = res.json()[0]['identifier']
year = YYYYMMDD[:4]
month = YYYYMMDD[4:6]
day = YYYYMMDD[6:8]
print(year, month, day)

# Basic getSat() by date

In [ ]:
## VERSION 1
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{YYYY}-{MM}-{DD}"
    res = requests.get(metadata, params=queryParams)
    print(res.status_code, res.url)

    # Construct the resource url
    img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"

    # Return json from requests' response body
    return res.json()

#getSat('nat', '2020', '04', '09')[1]

In [ ]:
## VERSION 02
# Define when
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    date = f"{YYYY}-{MM}-{DD}"
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    # Construct the resource url
    #img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"
    
    #Return relevant data as list
    return data

#getSat('nat', '2020', '03', '10')

In [ ]:
## VERSION 03
# Define when
def getSat(collection, img_type,YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the resource url
    if len(data) > 0:
        for e in data:
            print(e)
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            print(img)
    #Return relevant data as list
    return data

#getSat('nat', 'thumb', '2020', '03', '10')

In [2]:
## VERSION 04
def getSat(collection, img_type, YYYY,MM,DD, queryParams=dict()):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the image resource url
    satellite_images = []
    if len(data) > 0:
        for e in data:
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            satellite_images.append([img, e[1], len(res.json())])
            
    #Return relevant data as list
    return satellite_images

images = getSat('enh', 'thumb', '2020', '03', '01')

200 https://epic.gsfc.nasa.gov/api/enhanced/date/2020-03-01
There are 12 satellite images available for this date: 2020-03-01


In [7]:
## VERSION 05
def getSat(collection, img_type, YYYY,MM,DD, queryParams=dict()):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the image resource url
    satellite_images = []
    if len(data) > 0:
        for e in data:
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            satellite_images.append([img, e[1], len(res.json())])
            
    #Return relevant data as list
    return satellite_images

images = getSat('enh', 'thumb', '2020', '03', '01')

200 https://epic.gsfc.nasa.gov/api/enhanced/date/2020-03-01
There are 12 satellite images available for this date: 2020-03-01


In [ ]:
display(Image(url=images[0][0]))

# Working with cleaned dataset dates

In [4]:
import pandas as pd
import numpy as np
import datetime

In [5]:
df = pd.read_csv('OUTPUT/volcanic-explosions.csv')
df = df.drop(columns='Unnamed: 0')

In [ ]:
# Select an event on the dataframe for which we know there is available data in the API
event_date = df.iloc[20]['start']
event_date = event_date.split('-')
print(event_date)

# GET the API image and coordinates
available_data = getSat('nat', 'png', *event_date)
print(available_data[0])
print(available_data[-1])

# Reach API to get the links for each volcanic eruption in the dataset

In [10]:
# Access the START dates on each eruptive phase

def enrich_from_api():
    
    # What are we going to retrieve from each request?
    img_urls = []
    sat_lats = []
    sat_lons = []
    pics_that_day = []
    
    # Start looping the dataframe
    for phase_start in df['start']:
        print()
        print(phase_start)
        
        # Make the request to the API and store the data for a moment
        phase_data = getSat('nat', 'thumb', *phase_start.split('-'))
        print(phase_data)
        
        # Include the response data to the lists 
        if len(phase_data) > 0:
            print('This is the first available image for that date')
            
            # Indexes here are [0] because I only want to get the first image of the day
            # Possibly change the index to [-1] to get the last picture from that day 
            print(phase_data[0])
            img_urls.append(phase_data[0][0])
            sat_lats.append(phase_data[0][1]['lat'])
            sat_lons.append(phase_data[0][1]['lon'])
            pics_that_day.append(phase_data[0][2])
        else:
            img_urls.append('no-img')
            sat_lats.append('0')
            sat_lons.append('0')
            pics_that_day.append('0')
            
            
    # POSSIBLY INCLUDE A ZIP HERE ~
    
    return [img_urls, sat_lats, sat_lons, pics_that_day]


new_data = enrich_from_api()

df['start_img'] = new_data[0]
df['sat_lats'] = new_data[1]
df['sat_lons'] = new_data[2]
df['start_img_available_in_api'] = new_data[3]


2020-02-10
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-02-10
There are 0 satellite images available for this date: 2020-02-10
[]

2020-01-31
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-31
There are 0 satellite images available for this date: 2020-01-31
[]

2020-01-12
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-12
There are 0 satellite images available for this date: 2020-01-12
[]

2020-01-12
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-12
There are 0 satellite images available for this date: 2020-01-12
[]

2020-01-11
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-11
There are 0 satellite images available for this date: 2020-01-11
[]

2019-12-09
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-09
There are 0 satellite images available for this date: 2019-12-09
[]

2019-12-07
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-07
There are 0 satellite images available for this date: 2019-12-07
[]

2019-12-05
200 https://epi

200 https://epic.gsfc.nasa.gov/api/natural/date/2019-06-22
There are 22 satellite images available for this date: 2019-06-22
[['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622004555.jpg', {'lat': 20.357666, 'lon': 173.430176}, 22], ['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622015123.jpg', {'lat': 20.357666, 'lon': 157.038574}, 22], ['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622025650.jpg', {'lat': 20.29541, 'lon': 140.67627}, 22], ['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622040218.jpg', {'lat': 20.310059, 'lon': 124.313965}, 22], ['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622050746.jpg', {'lat': 20.324707, 'lon': 107.929688}, 22], ['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622061314.jpg', {'lat': 20.324707, 'lon': 91.625977}, 22], ['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumb

200 https://epic.gsfc.nasa.gov/api/natural/date/2019-05-16
There are 21 satellite images available for this date: 2019-05-16
[['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516002712.jpg', {'lat': 25.554199, 'lon': -177.890625}, 21], ['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516013240.jpg', {'lat': 25.539551, 'lon': 165.769043}, 21], ['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516023808.jpg', {'lat': 25.576172, 'lon': 149.39209}, 21], ['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516034336.jpg', {'lat': 25.568848, 'lon': 132.98584}, 21], ['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516044903.jpg', {'lat': 25.539551, 'lon': 116.638184}, 21], ['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516055431.jpg', {'lat': 25.532227, 'lon': 100.415039}, 21], ['https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thu

200 https://epic.gsfc.nasa.gov/api/natural/date/2019-04-09
There are 13 satellite images available for this date: 2019-04-09
[['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epic_1b_20190409000830.jpg', {'lat': 13.139648, 'lon': 176.967773}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epic_1b_20190409015633.jpg', {'lat': 13.205566, 'lon': 149.963379}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epic_1b_20190409034436.jpg', {'lat': 13.205566, 'lon': 122.958984}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epic_1b_20190409053239.jpg', {'lat': 13.293457, 'lon': 95.998535}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epic_1b_20190409072042.jpg', {'lat': 13.31543, 'lon': 68.972168}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epic_1b_20190409090845.jpg', {'lat': 13.395996, 'lon': 41.960449}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs

200 https://epic.gsfc.nasa.gov/api/natural/date/2019-02-16
There are 13 satellite images available for this date: 2019-02-16
[['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/thumbs/epic_1b_20190216005516.jpg', {'lat': -20.412598, 'lon': 162.202148}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/thumbs/epic_1b_20190216024318.jpg', {'lat': -20.335693, 'lon': 135.197754}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/thumbs/epic_1b_20190216043121.jpg', {'lat': -20.354004, 'lon': 108.156738}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/thumbs/epic_1b_20190216061924.jpg', {'lat': -20.280762, 'lon': 81.166992}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/thumbs/epic_1b_20190216080727.jpg', {'lat': -20.258789, 'lon': 54.09668}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/thumbs/epic_1b_20190216095529.jpg', {'lat': -20.20752, 'lon': 27.077637}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2019/02/16/t

...

200 https://epic.gsfc.nasa.gov/api/natural/date/2016-03-29
There are 13 satellite images available for this date: 2016-03-29
[['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1b_20160329000831.jpg', {'lat': 0.212402, 'lon': 169.973145}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1b_20160329015633.jpg', {'lat': 0.263672, 'lon': 142.954102}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1b_20160329034436.jpg', {'lat': 0.241699, 'lon': 115.927734}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1b_20160329053238.jpg', {'lat': 0.270996, 'lon': 88.894043}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1b_20160329072040.jpg', {'lat': 0.314941, 'lon': 61.82373}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1b_20160329090842.jpg', {'lat': 0.373535, 'lon': 34.82666}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2016/03/29/thumbs/epic_1

200 https://epic.gsfc.nasa.gov/api/natural/date/2016-01-08
There are 10 satellite images available for this date: 2016-01-08
[['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thumbs/epic_1b_20160108005516.jpg', {'lat': -23.950195, 'lon': 179.384766}, 10], ['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thumbs/epic_1b_20160108024318.jpg', {'lat': -23.90625, 'lon': 152.380371}, 10], ['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thumbs/epic_1b_20160108043121.jpg', {'lat': -23.942871, 'lon': 125.412598}, 10], ['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thumbs/epic_1b_20160108061924.jpg', {'lat': -23.950195, 'lon': 98.4375}, 10], ['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thumbs/epic_1b_20160108080726.jpg', {'lat': -23.95752, 'lon': 71.374512}, 10], ['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thumbs/epic_1b_20160108095529.jpg', {'lat': -23.950195, 'lon': 44.406738}, 10], ['https://epic.gsfc.nasa.gov/archive/natural/2016/01/08/thu

200 https://epic.gsfc.nasa.gov/api/natural/date/2015-11-12
There are 13 satellite images available for this date: 2015-11-12
[['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112010437.jpg', {'lat': -9.228516, 'lon': 155.544434}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112025239.jpg', {'lat': -9.272461, 'lon': 128.518066}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112044043.jpg', {'lat': -9.294434, 'lon': 101.513672}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112062846.jpg', {'lat': -9.338379, 'lon': 74.575195}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112081649.jpg', {'lat': -9.353027, 'lon': 47.585449}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112100452.jpg', {'lat': -9.42627, 'lon': 20.610352}, 13], ['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs

200 https://epic.gsfc.nasa.gov/api/natural/date/2015-09-29
There are 5 satellite images available for this date: 2015-09-29
[['https://epic.gsfc.nasa.gov/archive/natural/2015/09/29/thumbs/epic_1b_20150929054506.jpg', {'lat': 0.725098, 'lon': 84.023438}, 5], ['https://epic.gsfc.nasa.gov/archive/natural/2015/09/29/thumbs/epic_1b_20150929065034.jpg', {'lat': 0.791016, 'lon': 67.653809}, 5], ['https://epic.gsfc.nasa.gov/archive/natural/2015/09/29/thumbs/epic_1b_20150929075602.jpg', {'lat': 0.783691, 'lon': 51.276855}, 5], ['https://epic.gsfc.nasa.gov/archive/natural/2015/09/29/thumbs/epic_1b_20150929090129.jpg', {'lat': 0.695801, 'lon': 34.929199}, 5], ['https://epic.gsfc.nasa.gov/archive/natural/2015/09/29/thumbs/epic_1b_20150929100657.jpg', {'lat': 0.710449, 'lon': 18.684082}, 5]]
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2015/09/29/thumbs/epic_1b_20150929054506.jpg', {'lat': 0.725098, 'lon': 84.023438}, 5]

2015-09-23
200 https://epic.g

200 https://epic.gsfc.nasa.gov/api/natural/date/2015-07-07
There are 1 satellite images available for this date: 2015-07-07
[['https://epic.gsfc.nasa.gov/archive/natural/2015/07/07/thumbs/epic_1b_20150707192037.jpg', {'lat': 18.457031, 'lon': -99.42627}, 1]]
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2015/07/07/thumbs/epic_1b_20150707192037.jpg', {'lat': 18.457031, 'lon': -99.42627}, 1]

2015-06-29
200 https://epic.gsfc.nasa.gov/api/natural/date/2015-06-29
There are 0 satellite images available for this date: 2015-06-29
[]

2015-06-28
200 https://epic.gsfc.nasa.gov/api/natural/date/2015-06-28
There are 0 satellite images available for this date: 2015-06-28
[]

2015-06-24
200 https://epic.gsfc.nasa.gov/api/natural/date/2015-06-24
There are 0 satellite images available for this date: 2015-06-24
[]

2015-06-20
200 https://epic.gsfc.nasa.gov/api/natural/date/2015-06-20
There are 1 satellite images available for this date: 2015-06-20
[['http

In [ ]:
img_urls

In [12]:
df[20:30]

,v_num,v_name,erup_num,erup_cat,vei,start_y,start_m,start_d,end_y,end_m,end_d,lat,lon,start,end,delta,start_img,sat_lats,sat_lons,start_img_available_in_api
20,290250,Raikoke,22321,Confirmed Eruption,NaN,2019,6,22,2019,7,1,48.292,153.250,2019-06-22,2019-07-01,9 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,20.3577,173.43,22
21,233020,"Fournaise, Piton de la",22318,Confirmed Eruption,NaN,2019,6,11,2019,10,27,-21.244,55.708,2019-06-11,2019-10-27,138 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,22.6318,178.857,22
22,311120,Great Sitkin,22322,Confirmed Eruption,NaN,2019,6,1,2019,6,7,52.076,-176.130,2019-06-01,2019-06-07,6 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,24.2944,177.664,22
23,290240,Sarychev Peak,22320,Confirmed Eruption,2.0,2019,5,16,2019,10,7,48.092,153.200,2019-05-16,2019-10-07,144 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,25.5542,-177.891,21
24,341040,Colima,22323,Confirmed Eruption,NaN,2019,5,11,2019,7,12,19.514,-103.620,2019-05-11,2019-07-12,62 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,25.2686,171.98,22
25,282110,Asosan,22317,Confirmed Eruption,NaN,2019,4,16,2020,2,17,32.884,131.104,2019-04-16,2020-02-17,307 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,17.3584,178.857,13
26,300260,Klyuchevskoy,22310,Confirmed Eruption,NaN,2019,4,9,2019,7,2,56.056,160.642,2019-04-09,2019-07-02,84 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,13.1396,176.968,13
27,352090,Sangay,22312,Confirmed Eruption,NaN,2019,3,26,2020,2,19,-2.005,-78.341,2019-03-26,2020-02-19,330 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,3.20801,173.738,11
28,344020,San Cristobal,22311,Confirmed Eruption,NaN,2019,3,4,2019,3,4,12.702,-87.004,2019-03-04,2019-03-04,0 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,-12.085,170.178,13
29,233020,"Fournaise, Piton de la",22307,Confirmed Eruption,NaN,2019,2,18,2019,3,10,-21.244,55.708,2019-02-18,2019-03-10,20 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,-19.5337,171.255,13


In [19]:
df['sat_lons'].value_counts()

0             49
-179.94873     3
169.40918      2
155.874023     2
157.229004     2
              ..
175.041504     1
176.000977     1
177.663574     1
178.161621     1
167.299805     1
Name: sat_lons, Length: 126, dtype: int64

In [18]:
df.start_img_available_in_api.value_counts()

0     49
13    49
22    22
12    18
21    10
11     9
10     6
17     3
19     3
20     2
9      2
6      2
1      2
18     1
8      1
7      1
5      1
3      1
2      1
Name: start_img_available_in_api, dtype: int64

In [ ]:
0/0

In [13]:
df.to_csv('OUTPUT/enriched-data.csv')

In [ ]:
df[20:50]

In [ ]:
df = pd.DataFrame(date_rng, columns=['date'])
df['data'] = np.random.randint(0,100, size=(len(date_rng)))
df.head()

In [ ]:
date_rng = pd.date_range(start='1/1/2015', end='10/04/2020', freq='D')
date_rng

In [ ]:
df['datetime'] = pd.to_datetime(df.date)
df = df.set_index('datetime')

#list only the second day of each month
#df[df.index.day == 2]

# calculate mean of each year
df.resample('M').mean()